In [1]:
import numpy as np
from scipy.optimize import fsolve

def P_sat(T_K, A, B, C):
    """
    Calculate the saturation pressure (in bar) for a given temperature (in Kelvin)
    using the Antoine equation in the form:

        log10(P) = A - B/(T + C)

    where P is in bar and T is in Kelvin.

    Parameters:
      T_K : Temperature in Kelvin.
      A, B, C : Antoine parameters.

    Returns:
      Vapor pressure in bar.
    """
    return 10**(A - B/(T_K + C))

# -------------------------------------------------------------------------
# Functions to calculate bubble and dew point temperatures using Raoult's law
# -------------------------------------------------------------------------
def bubble_point_equation(T, z, P_total, A1, B1, C1, A2, B2, C2):
    """
    Bubble point equation for an ideal binary mixture (liquid feed):

         z*P_sat1(T) + (1-z)*P_sat2(T) = P_total

    where z is the mole fraction of component 1.

    Returns f(T) = z*P_sat1(T) + (1-z)*P_sat2(T) - P_total.
    """
    P1 = P_sat(T, A1, B1, C1)
    P2 = P_sat(T, A2, B2, C2)
    return z * P1 + (1 - z) * P2 - P_total

def dew_point_equation(T, z, P_total, A1, B1, C1, A2, B2, C2):
    """
    Dew point equation for an ideal binary mixture (vapor feed):

         z/P_sat1(T) + (1-z)/P_sat2(T) = 1/P_total

    where z is the mole fraction of component 1 in the overall mixture.

    Returns f(T) = z/P_sat1(T) + (1-z)/P_sat2(T) - 1/P_total.
    """
    P1 = P_sat(T, A1, B1, C1)
    P2 = P_sat(T, A2, B2, C2)
    return z / P1 + (1 - z) / P2 - 1 / P_total

def main():
    # ------------------------------------------------------------
    # Feed and system parameters for the distillation column feed
    # ------------------------------------------------------------
    # Total system pressure (bar)
    P_total = 1.0  # bar

    # Overall feed composition (mole fraction of component 1)
    z = 0.4  # For example, 40% component 1, 60% component 2

    # Antoine parameters for Component 1 (e.g., water-like)
    # These parameters are given in a form valid for T (K) and P (bar)
    A1 = 4.00266
    B1 = 1171.53
    C1 = -48.784

    # Antoine parameters for Component 2 (e.g., ethanol-like)
    A2 = 4.02832
    B2 = 1268.636
    C2 = -56.199

    # Feed stream temperature (actual feed condition in Kelvin)
    T_feed = 300.15  # K (modify this value to match your process)

    # ------------------------------------------------------------
    # Solve for bubble and dew point temperatures (ideal system using Raoult's law)
    # ------------------------------------------------------------
    # Provide an initial guess for temperature (in Kelvin)
    T_guess = 373.15  # A reasonable starting guess

    # Solve bubble point: find T such that bubble_point_equation(T) = 0
    T_bubble, = fsolve(bubble_point_equation, T_guess, args=(z, P_total, A1, B1, C1, A2, B2, C2))

    # Solve dew point: find T such that dew_point_equation(T) = 0
    T_dew, = fsolve(dew_point_equation, T_guess, args=(z, P_total, A1, B1, C1, A2, B2, C2))

    # ------------------------------------------------------------
    # Determine the feed condition based on T_feed relative to T_bubble and T_dew
    # ------------------------------------------------------------
    if T_feed < T_bubble:
        feed_condition = "Subcooled Liquid (feed temperature below bubble point)"
    elif T_feed > T_dew:
        feed_condition = "Superheated Vapor (feed temperature above dew point)"
    else:
        feed_condition = "Saturated (two-phase) feed"

    # ------------------------------------------------------------
    # Output the results
    # ------------------------------------------------------------
    print("Feed Binary Mixture (entering a distillation column):")
    print("  Overall feed composition (z): {:.2f}".format(z))
    print("  Total system pressure: {:.2f} bar".format(P_total))
    print("")
    print("Calculated Temperatures:")
    print("  Bubble point temperature = {:.2f} K".format(T_bubble))
    print("  Dew point temperature    = {:.2f} K".format(T_dew))
    print("  Feed stream temperature  = {:.2f} K".format(T_feed))
    print("")
    print("Feed condition: " + feed_condition)

if __name__ == "__main__":
    main()


Feed Binary Mixture (entering a distillation column):
  Overall feed composition (z): 0.40
  Total system pressure: 1.00 bar

Calculated Temperatures:
  Bubble point temperature = 356.19 K
  Dew point temperature    = 362.33 K
  Feed stream temperature  = 300.15 K

Feed condition: Subcooled Liquid (feed temperature below bubble point)
